Here we are using pandas ai library to visualize data with prompt(without code)

In [ ]:
pip install pandasai


In [3]:
import pandas as pd
from pandasai import SmartDataframe

# Sample DataFrame
sales_by_country = pd.read_csv('D:/git projects/data chat bot/Prompt-Based-Data-Analysis-and-Visualization-Chatbot-for-CSV-Data/car_price_dataset.csv')

# Instantiate a LLM
from pandasai.llm import OpenAI
import os
openai_api_key = os.environ["API_KEY"]

llm = OpenAI(api_token=openai_api_key)

df = SmartDataframe(sales_by_country, config={"llm": llm})
df.chat('show rows with null values')

,Company Name,Model Name,Price,Model Year,Location,Mileage,Engine Type,Engine Capacity,Color,Assembly,Body Type,Transmission Type,Registration Status


From here training the pytorch model for prediction

In [3]:
class basic_checks():
    
    def read_data(self, data_path):
        
        return pd.read_csv(data_path)

bc=basic_checks()
df=bc.read_data('C:/Users/vijju/OneDrive/Desktop/Projects/WP/FINAL DOCS WP/weather1.csv')

In [4]:
df.head()

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True


In [5]:
def convert_date(df):
     
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['month']=df['DATE'].dt.month
    df['day']=df['DATE'].dt.day
    return df

df=convert_date(df)
print(df.head())

        DATE  PRCP  TMAX  TMIN  RAIN  month  day
0 1948-01-01  0.47    51    42  True      1    1
1 1948-01-02  0.59    45    36  True      1    2
2 1948-01-03  0.42    45    35  True      1    3
3 1948-01-04  0.31    45    34  True      1    4
4 1948-01-05  0.17    45    32  True      1    5


In [7]:
df=df.drop(['DATE'],axis=1)

In [10]:
df.head()

,day,month,PRCP,TMAX,TMIN,RAIN
0,1,1,0.47,51,42,True
1,2,1,0.59,45,36,True
2,3,1,0.42,45,35,True
3,4,1,0.31,45,34,True
4,5,1,0.17,45,32,True


In [9]:
df=df.reindex(columns=['day','month','PRCP','TMAX','TMIN','RAIN'])

In [13]:
df.isnull().sum()

day      0
month    0
PRCP     0
TMAX     0
TMIN     0
RAIN     0
dtype: int64

In [12]:
df.dropna(inplace=True)

In [15]:
df['RAIN'] = df['RAIN'].apply(lambda x: 1 if x == True else 0)

In [16]:
df.head()

,day,month,PRCP,TMAX,TMIN,RAIN
0,1,1,0.47,51,42,1
1,2,1,0.59,45,36,1
2,3,1,0.42,45,35,1
3,4,1,0.31,45,34,1
4,5,1,0.17,45,32,1


In [17]:
x=df.drop(["RAIN"],axis=1)
y=df["RAIN"]
print(x)
print(y)

       day  month  PRCP  TMAX  TMIN
0        1      1  0.47    51    42
1        2      1  0.59    45    36
2        3      1  0.42    45    35
3        4      1  0.31    45    34
4        5      1  0.17    45    32
...    ...    ...   ...   ...   ...
25546   10     12  0.00    49    34
25547   11     12  0.00    49    29
25548   12     12  0.00    46    32
25549   13     12  0.00    48    34
25550   14     12  0.00    50    36

[25548 rows x 5 columns]
0        1
1        1
2        1
3        1
4        1
        ..
25546    0
25547    0
25548    0
25549    0
25550    0
Name: RAIN, Length: 25548, dtype: int64


In [27]:
# report which features were selected by RFE
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
# define RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=1)
# fit RFE
rfe.fit(x, y)
# summarize all features
for i in range(x.shape[1]):
 print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))


Column: 0, Selected=False, Rank: 5
Column: 1, Selected=False, Rank: 4
Column: 2, Selected=True, Rank: 1
Column: 3, Selected=False, Rank: 3
Column: 4, Selected=False, Rank: 2


In [28]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0,stratify=y)

In [30]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

#scaler.fit()


xtrain=scaler.fit_transform(x_train)
xtest=scaler.fit_transform(x_test)

In [ ]:
'''import xgboost as xgb

## Define hyperparameters
#params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 5}
#
## Convert categorical features to DMatrix format
#dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
#dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# Train the classifier
model = xgb.train(xtrain,y_train)'''

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.tree import DecisionTreeClassifier,plot_tree
model_dt=DecisionTreeClassifier(random_state=0)
model_dt.fit(xtrain,y_train)

prediction=model_dt.predict(xtest)

print(accuracy_score(y_test,prediction))
print(confusion_matrix(y_test,prediction))
print("classification_report:")
print(classification_report(y_test,prediction))

1.0
[[2930    0]
 [   0 2180]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2930
           1       1.00      1.00      1.00      2180

    accuracy                           1.00      5110
   macro avg       1.00      1.00      1.00      5110
weighted avg       1.00      1.00      1.00      5110



In [40]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(xtrain, y_train)
prediction=knn.predict(xtest)
print(accuracy_score(y_test,prediction))
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))

0.900587084148728
[[2877   53]
 [ 455 1725]]
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      2930
           1       0.97      0.79      0.87      2180

    accuracy                           0.90      5110
   macro avg       0.92      0.89      0.90      5110
weighted avg       0.91      0.90      0.90      5110



In [43]:
#this give more acc if dont do normalization
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x_train,y_train)
prediction=gnb.predict(x_test)
print(accuracy_score(y_test,prediction))
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))



1.0
[[2930    0]
 [   0 2180]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2930
           1       1.00      1.00      1.00      2180

    accuracy                           1.00      5110
   macro avg       1.00      1.00      1.00      5110
weighted avg       1.00      1.00      1.00      5110



In [45]:
from sklearn.svm import SVC
svc=SVC(kernel="linear")
svc.fit(xtrain,y_train)
prediction=svc.predict(xtest)
print(accuracy_score(y_test,prediction))
print(confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))

1.0
[[2930    0]
 [   0 2180]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2930
           1       1.00      1.00      1.00      2180

    accuracy                           1.00      5110
   macro avg       1.00      1.00      1.00      5110
weighted avg       1.00      1.00      1.00      5110



In [46]:
import numpy as np
input_data=(12,	12,	0.00,	46,	32			)
np_array=np.asarray(input_data)
reshaped=np_array.reshape(-1,1)
data=scaler.fit_transform(reshaped)

data

array([[-0.51173869],
       [-0.51173869],
       [-1.24279396],
       [ 1.55958457],
       [ 0.70668676]])

In [47]:

data=data.reshape(1,-1)
data

prediction=svc.predict(data)

In [48]:
prediction
if prediction == 0:
  print("false")
else:
  print("true")

false


additional code
